## ASSIGNMENT 04 : BASIC STATS 02

Problem Statement : Hospital Patient Data Analysis Context: A hospital maintains patient records including admission details, department, diagnosis, doctor, and bill amount. You have two datasets: one with patient info and another with billing details. Some patients have blank bill amounts, and there are multiple rows for the same patient due to follow-ups. Tasks:

* Load the patient dataset and show summary with info().
* Select only the columns relevant for billing: ['PatientID', 'Department', 'Doctor', 'BillAmount'].
* Drop administrative columns like ['ReceptionistID', 'CheckInTime'].
* Use groupby to find total bill amount per department.
* Remove duplicate patient records based on PatientID.
* Fill missing BillAmount values with the mean bill amount.
* Merge the billing dataset with patient dataset on PatientID.
* Concatenate an additional DataFrame that contains new patients for the current week (row-wise).
* Concatenate new billing category columns like ['InsuranceCovered', 'FinalAmount'] (column-wise).
 
Expected Outcome: 
* Final cleaned dataset with accurate billing info. 
* All missing values handled, merged dataset across PatientID.
* Ability to perform further analytics on department-wise revenue or doctor performance.

In [1]:
import pandas as pd

In [7]:

df_patient = pd.read_csv('Patient_Data.csv')
df_patient

,PatientID,Name,Department,Doctor,BillAmount,ReceptionistID,CheckInTime
0,101,Alice,Cardiology,Dr. Smith,5000.0,1,2023-01-10 09:00
1,102,Bob,Neurology,Dr. John,NaN,2,2023-01-11 10:30
2,103,Charlie,Orthopedics,Dr. Lee,7500.0,1,2023-01-12 11:00
3,104,David,Cardiology,Dr. Smith,6200.0,3,2023-01-13 12:00
4,105,Eva,Dermatology,Dr. Rose,NaN,2,2023-01-14 08:45
5,101,Alice,Cardiology,Dr. Smith,5000.0,1,2023-01-10 09:00


In [8]:
df_patient.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PatientID       6 non-null      int64  
 1   Name            6 non-null      object 
 2   Department      6 non-null      object 
 3   Doctor          6 non-null      object 
 4   BillAmount      4 non-null      float64
 5   ReceptionistID  6 non-null      int64  
 6   CheckInTime     6 non-null      object 
dtypes: float64(1), int64(2), object(4)
memory usage: 468.0+ bytes


In [13]:
df_patient_billing = df_patient[['PatientID','Department','Doctor','BillAmount']]
df_patient_billing.head()

,PatientID,Department,Doctor,BillAmount
0,101,Cardiology,Dr. Smith,5000.0
1,102,Neurology,Dr. John,NaN
2,103,Orthopedics,Dr. Lee,7500.0
3,104,Cardiology,Dr. Smith,6200.0
4,105,Dermatology,Dr. Rose,NaN


In [23]:
df_clean_admin = df_patient.drop(['ReceptionistID', 'CheckInTime'], axis=1, errors='ignore')

In [24]:
df_clean_admin.head()

,PatientID,Name,Department,Doctor,BillAmount
0,101,Alice,Cardiology,Dr. Smith,5000.0
1,102,Bob,Neurology,Dr. John,NaN
2,103,Charlie,Orthopedics,Dr. Lee,7500.0
3,104,David,Cardiology,Dr. Smith,6200.0
4,105,Eva,Dermatology,Dr. Rose,NaN


In [27]:
df_dept_billing = df_patient.groupby('Department')['BillAmount'].sum()
df_dept_billing

Department
Cardiology     16200.0
Dermatology        0.0
Neurology          0.0
Orthopedics     7500.0
Name: BillAmount, dtype: float64

In [28]:
df_no_duplicate = df_patient.drop_duplicates(subset=['PatientID'])
df_no_duplicate

,PatientID,Name,Department,Doctor,BillAmount
0,101,Alice,Cardiology,Dr. Smith,5000.0
1,102,Bob,Neurology,Dr. John,NaN
2,103,Charlie,Orthopedics,Dr. Lee,7500.0
3,104,David,Cardiology,Dr. Smith,6200.0
4,105,Eva,Dermatology,Dr. Rose,NaN


In [41]:
mean_bill = df_no_duplicate['BillAmount'].mean()  

# Fill missing
df_filled = df_no_duplicate.copy()
df_filled['BillAmount'] = df_filled['BillAmount'].fillna(mean_bill)

df_filled.head()

,PatientID,Name,Department,Doctor,BillAmount
0,101,Alice,Cardiology,Dr. Smith,5000.000000
1,102,Bob,Neurology,Dr. John,6233.333333
2,103,Charlie,Orthopedics,Dr. Lee,7500.000000
3,104,David,Cardiology,Dr. Smith,6200.000000
4,105,Eva,Dermatology,Dr. Rose,6233.333333


In [47]:
df_billing = pd.read_csv('Billing_Data.csv')

df_merged = pd.merge(df_filled,df_billing,on='PatientID', how='inner')
df_merged.head()


,PatientID,Name,Department,Doctor,BillAmount,InsuranceCovered,FinalAmount
0,101,Alice,Cardiology,Dr. Smith,5000.000000,2000,3000
1,102,Bob,Neurology,Dr. John,6233.333333,1500,3500
2,103,Charlie,Orthopedics,Dr. Lee,7500.000000,2500,5000
3,104,David,Cardiology,Dr. Smith,6200.000000,3000,3200
4,105,Eva,Dermatology,Dr. Rose,6233.333333,1000,4000


In [49]:
new_patients = pd.DataFrame({
    'PatientID': [2000,2001],
    'Department': ['Orthopedics','Neurology'],
    'Doctor': ['Dr.X','Dr.Y'],
    'BillAmount': [6000,4000]
})

added_df = pd.concat([df_merged, new_patients],ignore_index=True)
added_df.tail()


,PatientID,Name,Department,Doctor,BillAmount,InsuranceCovered,FinalAmount
2,103,Charlie,Orthopedics,Dr. Lee,7500.000000,2500.0,5000.0
3,104,David,Cardiology,Dr. Smith,6200.000000,3000.0,3200.0
4,105,Eva,Dermatology,Dr. Rose,6233.333333,1000.0,4000.0
5,2000,NaN,Orthopedics,Dr.X,6000.000000,NaN,NaN
6,2001,NaN,Neurology,Dr.Y,4000.000000,NaN,NaN


In [51]:

new_cols = pd.DataFrame({
    'InsuranceCovered': (['Yes', 'No'] * len(added_df))[:len(added_df)],
    'FinalAmount': added_df['BillAmount'] * 0.9
})

final_df = pd.concat([added_df, new_cols], axis=1)
final_df.head()

,PatientID,Name,Department,Doctor,BillAmount,InsuranceCovered,FinalAmount,InsuranceCovered,FinalAmount
0,101,Alice,Cardiology,Dr. Smith,5000.000000,2000.0,3000.0,Yes,4500.0
1,102,Bob,Neurology,Dr. John,6233.333333,1500.0,3500.0,No,5610.0
2,103,Charlie,Orthopedics,Dr. Lee,7500.000000,2500.0,5000.0,Yes,6750.0
3,104,David,Cardiology,Dr. Smith,6200.000000,3000.0,3200.0,No,5580.0
4,105,Eva,Dermatology,Dr. Rose,6233.333333,1000.0,4000.0,Yes,5610.0


In [52]:
final_df.head()

,PatientID,Name,Department,Doctor,BillAmount,InsuranceCovered,FinalAmount,InsuranceCovered,FinalAmount
0,101,Alice,Cardiology,Dr. Smith,5000.000000,2000.0,3000.0,Yes,4500.0
1,102,Bob,Neurology,Dr. John,6233.333333,1500.0,3500.0,No,5610.0
2,103,Charlie,Orthopedics,Dr. Lee,7500.000000,2500.0,5000.0,Yes,6750.0
3,104,David,Cardiology,Dr. Smith,6200.000000,3000.0,3200.0,No,5580.0
4,105,Eva,Dermatology,Dr. Rose,6233.333333,1000.0,4000.0,Yes,5610.0
